In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} 3D visualization of Riemann sheets
TR-997
^^^
Follow-up to [TR-004](004.ipynb), where we investigate and reproduce the Riemann sheets shown in [Fig.&nbsp;50.1](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=2) and [50.2](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=4) of the PDG.
:::
::::

# Riemann sheets for two channels

In [ ]:
%pip install -q ampform==0.14.8 plotly==5.18.0 sympy==1.12

In [ ]:
from __future__ import annotations

import warnings
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import unevaluated
from IPython.display import Math

warnings.filterwarnings("ignore")

## Phase space factors

In [ ]:
@unevaluated(real=False)
class SignedSqrt(sp.Expr):
    z: Any
    _latex_repr_ = R"\sqrt[+]{{{z}}}"

    def evaluate(self) -> sp.Expr:
        z = self.args[0]
        return sp.sqrt(abs(z)) * sp.exp(sp.I * PosArg(z) / 2)


@unevaluated
class PosArg(sp.Expr):
    z: Any
    _latex_repr_ = R"\arg^+\left({z}\right)"

    def evaluate(self) -> sp.Expr:
        z = self.args[0]
        arg = sp.arg(z)
        return sp.Piecewise(
            (arg - 2 * sp.pi, arg < -2 * sp.pi),
            (arg + 2 * sp.pi, arg > 0),
            (arg, True),
        )


z = sp.Symbol("z", complex=True)
Math(aslatex({e: e.evaluate() for e in [SignedSqrt(z), PosArg(z)]}))

In [ ]:
@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^+_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return SignedSqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)


s, m1, m2 = sp.symbols("s m1 m2")
rho_expr = PhaseSpaceFactor(s, m1, m2)
Math(aslatex({rho_expr: rho_expr.doit(deep=False)}))

Phase space factor with Chew-Mandelstam for $S$ waves, from [PDG 2023, Section Resonances, p.15](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=15):

In [ ]:
@unevaluated(real=False)
class PhaseSpaceCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}_{{{m1},{m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m_a: Any
    m_b: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m_a, m_b = self.args
        return SignedSqrt((s - (m_a + m_b) ** 2) * (s - (m_a - m_b) ** 2) / (s * 4))


rho_cm_expr = PhaseSpaceCM(s, m1, m2)
cm_expr = ChewMandelstam(s, m1, m2)
q_expr = BreakupMomentum(s, m1, m2)
Math(aslatex({e: e.doit(deep=False) for e in [rho_cm_expr, cm_expr, q_expr]}))

### Visual comparison

In [ ]:
%config InlineBackend.figure_formats = ['svg']
symbols = sp.Tuple(s, m1, m2)
cm_func = sp.lambdify(symbols, cm_expr.doit())
rho_func = sp.lambdify(symbols, rho_expr.doit())

x = np.linspace(0, 2, 400)
values = {
    m1: 0.6,
    m2: 0.4,
}
args = eval(str(symbols[1:].subs(values)))

cm_values = 16 * np.pi * cm_func(x, *args)
rho_values = 1j * rho_func(x, *args)

fig, axes = plt.subplots(figsize=(15, 6), ncols=2, sharex=True, sharey=True)
ax1, ax2 = axes
for ax in axes:
    ax.set_xlabel(R"$\mathrm{Re}(s)$")
ax1.set_ylabel("Function value")

ax1.plot(x, cm_values.real, label="Real part")
ax1.plot(x, cm_values.imag, label="Imaginary part")
ax1.set_title("Chew Mandelstam Function")
ax1.legend()

ax2.plot(x, rho_values.real, label="Real part")
ax2.plot(x, rho_values.imag, label="Imaginary part")
ax2.set_title("Phase Space Factor")
ax2.legend()

ax2.set_ylim(-1.2, +1.6)
fig.tight_layout()

plt.show()

## Define parameters values for plotting 

## T matrix definition with K matrix

In [ ]:
class RhoMatrix(sp.DiagonalMatrix):
    def _latex(self, printer):
        return R"\rho"


n_channels = 2
I = sp.Identity(n_channels)
K = sp.MatrixSymbol("K", n_channels, n_channels)
rho = RhoMatrix(sp.MatrixSymbol("rho", n_channels, n_channels))
Math(aslatex({rho: rho.as_explicit()}))

In [ ]:
T = (I - sp.I * K * rho).inv() * K
T

In [ ]:
T_explicit = T.as_explicit()
T_explicit

:::{note}
No Blatt-Weisskopf form factors because $L=0$.
:::

In [ ]:
s = sp.Symbol("s")
ma1 = sp.Symbol("m_{a1}")
mb1 = sp.Symbol("m_{b1}")
ma2 = sp.Symbol("m_{a2}")
mb2 = sp.Symbol("m_{b2}")
m0 = sp.Symbol("m0")
w0 = sp.Symbol("Gamma0")
g1 = sp.Symbol("gamma1")
g2 = sp.Symbol("gamma2")

k_expr_00 = (g1 * g1 * m0 * w0) / (s - m0**2)
k_expr_10 = (g1 * g2 * m0 * w0) / (s - m0**2)
k_expr_11 = (g2 * g2 * m0 * w0) / (s - m0**2)

In [ ]:
rho_expressions = {
    K[0, 0]: k_expr_00,
    K[1, 1]: k_expr_11,
    K[0, 1]: k_expr_10,
    K[1, 0]: k_expr_10,
    rho[0, 0]: PhaseSpaceFactor(s, ma1, mb1),
    rho[1, 1]: PhaseSpaceFactor(s, ma2, mb2),
}
cm_expressions = {
    **rho_expressions,
    rho[0, 0]: PhaseSpaceCM(s, ma1, mb1),
    rho[1, 1]: PhaseSpaceCM(s, ma2, mb2),
}
Math(aslatex(cm_expressions))

In [ ]:
cm_conj1_expressions = {
    **cm_expressions,
    rho[0, 0]: sp.I * 16 * sp.pi * ChewMandelstam(s, ma1, mb1).conjugate(),
    rho[1, 1]: sp.I * 16 * sp.pi * ChewMandelstam(s, ma2, mb2),
}
cm_conj2_expressions = {
    **cm_expressions,
    rho[0, 0]: sp.I * 16 * sp.pi * ChewMandelstam(s, ma1, mb1),
    rho[1, 1]: sp.I * 16 * sp.pi * ChewMandelstam(s, ma2, mb2).conjugate(),
}
cm_conj_conj_expressions = {
    **cm_expressions,
    rho[0, 0]: sp.I * 16 * sp.pi * ChewMandelstam(s, ma1, mb1).conjugate(),
    rho[1, 1]: sp.I * 16 * sp.pi * ChewMandelstam(s, ma2, mb2).conjugate(),
}

In [ ]:
T_cm_expr = T_explicit.xreplace(cm_expressions)
T_cm_conj1_expr = T_explicit.xreplace(cm_conj1_expressions)
T_cm_conj2_expr = T_explicit.xreplace(cm_conj2_expressions)
T_cm_conj_conj_expr = T_explicit.xreplace(cm_conj_conj_expressions)

In [ ]:
T_cm_expr[0, 0].simplify(doit=False)

## T Matrix

In [ ]:
symbols = sp.Tuple(s, ma1, mb1, ma2, mb2, m0, w0, g1, g2)
T_cm_func_00 = sp.lambdify(symbols, T_cm_expr[0, 0].doit())
T_cm_conj1_func_00 = sp.lambdify(symbols, T_cm_conj1_expr[0, 0].doit())
T_cm_conj2_func_00 = sp.lambdify(symbols, T_cm_conj2_expr[0, 0].doit())
T_cm_conj_conj_func_00 = sp.lambdify(symbols, T_cm_conj_conj_expr[0, 0].doit())

In [ ]:
values = {
    ma1: 0.08,
    mb1: 0.12,
    ma2: 0.25,
    mb2: 0.35,
    m0: 1.0,
    w0: 0.5,
    g1: 1,
    g2: 1,
}
args = eval(str(symbols[1:].xreplace(values)))

In [ ]:
epsilon = 1e-5
x = np.linspace(0, 4.0, num=400)
y = np.linspace(epsilon, 1.0, num=400)
X, Y = np.meshgrid(x, y)
Zn = X - Y * 1j
Zp = X + Y * 1j

Tp_cm_00 = T_cm_func_00(Zp, *args)
Tn_cm_00 = T_cm_func_00(Zn, *args)

Tp_cm_conj1_00 = T_cm_conj1_func_00(Zp, *args)
Tn_cm_conj1_00 = T_cm_conj1_func_00(Zn, *args)

Tp_cm_conj2_00 = T_cm_conj2_func_00(Zp, *args)
Tn_cm_conj2_00 = T_cm_conj2_func_00(Zn, *args)

Tp_cm_conj_conj_00 = T_cm_conj_conj_func_00(Zp, *args)
Tn_cm_conj_conj_00 = T_cm_conj_conj_func_00(Zn, *args)

In [ ]:
%config InlineBackend.figure_formats = ['svg']
thr1 = ma1 + mb1
thr2 = ma2 + mb2
fig, ax = plt.subplots()
ax.set_xlabel(R"$m=\sqrt{s}$")
ax.set_ylabel(R"$\mathrm{Im}\,T$")
ax.plot(
    np.sqrt(x),
    Tp_cm_00[0].imag,
    c="red",
    label="$T_{00}(s+0i)$",
)
ax.plot(
    np.sqrt(x),
    Tp_cm_conj1_00[0].imag,
    c="orange",
    label="$T_{00}(s+0i)$ conj 1",
)
ax.plot(
    np.sqrt(x),
    Tn_cm_conj2_00[0].imag,
    c="green",
    ls="dotted",
    label="$T_{00}(s-0i)$ conj 2",
)
ax.plot(
    np.sqrt(x),
    Tp_cm_conj_conj_00[0].imag,
    c="blue",
    label="$T_{00}(s+0i)$ conj 1,2",
)
ax.axvline(m0.subs(values), ls="dashed", c="black", label=f"${sp.latex(m0)}$")
ax.axvline(thr1.subs(values), ls="dotted", c="C1", label=f"${sp.latex(thr1)}$")
ax.axvline(thr2.subs(values), ls="dotted", c="C2", label=f"${sp.latex(thr2)}$")
ax.set_ylim(-0.1, +1.5)
ax.legend()
fig.tight_layout()
plt.show()

In [ ]:
%config InlineBackend.figure_formats = ['png']
fig, axes = plt.subplots(
    ncols=2, nrows=2, figsize=(12, 10), sharex=True, sharey=True
)
fig.suptitle("$\mathrm{Im}\,T$")
for ax in axes[1, :]:
    ax.set_xlabel(R"$\mathrm{Re}\,s$")
for ax in axes[:, 0]:
    ax.set_ylabel(R"$\mathrm{Re}\,S$")
T_max = 1.5
kwargs = dict(
    cmap=plt.cm.coolwarm,
    vmin=-T_max,
    vmax=+T_max,
)

rho1 = PhaseSpaceCM(s, ma1, mb1)
rho2 = PhaseSpaceCM(s, ma2, mb2)
axes[0, 0].set_title(f"${sp.latex(rho1)}, {sp.latex(rho2)}$")
axes[0, 0].pcolormesh(X, -Y, Tn_cm_00.imag, **kwargs)
axes[0, 0].pcolormesh(X, +Y, Tp_cm_00.imag, **kwargs)
axes[0, 1].set_title(f"${sp.latex(rho1)}, {sp.latex(rho2.conjugate())}$")
axes[0, 1].pcolormesh(X, -Y, Tn_cm_conj1_00.imag, **kwargs)
axes[0, 1].pcolormesh(X, +Y, Tp_cm_conj1_00.imag, **kwargs)
axes[1, 0].set_title(f"${sp.latex(rho1.conjugate())}, {sp.latex(rho2)}$")
axes[1, 0].pcolormesh(X, -Y, Tn_cm_conj2_00.imag, **kwargs)
axes[1, 0].pcolormesh(X, +Y, Tp_cm_conj2_00.imag, **kwargs)
axes[1, 1].set_title(f"${sp.latex(rho1.conjugate())}, {sp.latex(rho2.conjugate())}$")
axes[1, 1].pcolormesh(X, -Y, Tn_cm_conj_conj_00.imag, **kwargs)
axes[1, 1].pcolormesh(X, +Y, Tp_cm_conj_conj_00.imag, **kwargs)

for ax in axes.flatten():
    ax.axvline(m0.subs(values) ** 2, ls="dashed", c="black")
    ax.axvline(thr1.subs(values) ** 2, ls="dotted", c="C1")
    ax.axvline(thr2.subs(values) ** 2, ls="dotted", c="C2")

fig.tight_layout()
plt.show()